In [30]:
degBit = 11
degStable = 11
delta = 1/6

def binomial_pdf(n, p, x):
    return binomial(n, x) * p ^ x * (1 - p) ^ (n-x)

# Pr[X >= x]
def binomial_ncdf(n, p, x):
    probSum = 0.0
    for i in range(x, n + 1):
        probSum = probSum + binomial_pdf(n, p, i)
    return probSum

# phi(targetP) = Pr[X >= x], return x
def binomial_ncdf_inverse(n, p, targetP):
    for i in range(1, n):
        ncdf_p = binomial_ncdf(n, p, i)
        if ncdf_p < targetP:
            return i - 1
    return n


# https://stats.stackexchange.com/questions/394960/variance-of-normal-order-statistics
# Variance for quantile
# TODO: verify
def binomial_inverse_variance(N, p, inverse, M):
	return p * (1-p) / ((M+2) * (binomial_pdf(N, p, inverse)^2))

## Lower bound via taking advantage of discrete RV
# where V < mu
def chebyshevLowerBound(V, mu, variance):
	return min(variance / (mu - V)^2, 1)

## Upper bound via taking advantage of discrete RV
# where V > mu and we are looking for Pr[mu < V]
def chebyshevUpperBound(V, mu, variance):
	return 1 - min(variance / (mu - V)^2, 0)

def pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG):
	# a lower bound for the expectation of G
	nStabilizersForG = degBit * degStable - degStable * E
	if nStabilizersForG <= 0:
		return 1
	expectedG = binomial_ncdf_inverse(nStabilizersForG, pG, k / M)
	varianceG = binomial_inverse_variance(nStabilizersForG, pG, expectedG, M)
	# print(expectedG, varianceG, pG, nStabilizersForG)

	expectedGPrime = expectedG * (1 - delta)
	varianceGPrime = varianceG * (1 - delta) ^ 0.5

	gPrimeForError = (3 * degBit * E + E - delta * E) / (3 * degBit - 1 + delta)
	
	# Note: this gives an upper bound for error
	# Just return the probability that Pr[G'
	if gPrimeForError == expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError + 1, expectedGPrime, varianceGPrime)
	elif gPrimeForError > expectedGPrime:
		return 1#chebyshevUpperBound(gPrimeForError, expectedGPrime, varianceGPrime)
	else:
		return chebyshevLowerBound(gPrimeForError, expectedGPrime, varianceGPrime)

def pr_G_and_E_error(n_E, p_E, E, degBit, delta, k, M, pG):
	prE = binomial_pdf(n_E, p_E, E)
	return prE * pr_G_error_given_E(E, degBit, degStable, delta, k, M, pG)


def calc_pr_indep_G_E_lt_v_for_k(degBit, stableDeg, delta, v, k, pError, nE, pE, pG, M):
	maxE = nE
	prLtV = 0
	for e in range(0, maxE):
		prLtV = prLtV + pr_G_and_E_error(nE, pE, e, degBit, delta, k, M, pG)
	return prLtV

def calc_pr_indep_G_E_lt_v(bitDeg, stableDeg, delta, v, kRange, pError, M):
	pG = 0.5 - 0.5 * (1 - 2 * pError)^stableDeg
	pE = 0.5 + 0.5 * (1 - 2 * pError)^stableDeg - (1 - pError)^stableDeg
	nG = floor(bitDeg * stableDeg)
	nE = floor(bitDeg * stableDeg)

	error_prob = 1
	for k in range(*kRange):
		tmp = calc_pr_indep_G_E_lt_v_for_k(bitDeg, stableDeg, delta, v, k, pError, nE, pE, pG, M)
		print(k, tmp)
		error_prob = error_prob * tmp
	return error_prob

# TODO: make some plots?? (After LaTeX to ensure that I have some soundness to my arguments)
error_prob = calc_pr_indep_G_E_lt_v(degBit, degStable, delta, 1/6, (1, 100), 0.05, 1000)
error_prob

1 0.954563699858651
2 0.803410999935081
3 0.799487844470336
4 0.794746764921984
5 0.785490914524109
6 0.760831763689193
7 0.760112337769818
8 0.760112245782353
9 0.758872674465862
10 0.758872674465862
11 0.756887512333625
12 0.756868960382458
13 0.756673408859769
14 0.751805859576252
15 0.751803220439395
16 0.751803220439395
17 0.751450816898134
18 0.751450816898134
19 0.751450801357154
20 0.751443951550248
21 0.751443951550248
22 0.751443951550248
23 0.750854998974413
24 0.750853943127446
25 0.750853943127446
26 0.750853943127446
27 0.750853825283121
28 0.750840048265148
29 0.750840048265148
30 0.750840041283354
31 0.750840041283354
32 0.749537906322160
33 0.749535160632910
34 0.749535160632910
35 0.749535160632910
36 0.749535160632910
37 0.749535160632910
38 0.749534707261149
39 0.749534707261149
40 0.749511326986260
41 0.749511273473815
42 0.749511273473815
43 0.749511273473815
44 0.749511273473815
45 0.749511270152676
46 0.749348794602630
47 0.749343260109104
48 0.749343260109104
4

7.20945322980022e-13